In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
from path import Path

from src.config import LOG_DIR
from src.utils import read_json_log_file

LOG_DIR = "notebooks/data/kaggle_single_out_1000/logs"

# Specify the file path
file_path = Path(parent_dir) / LOG_DIR / "plain/logs.json"

logs = read_json_log_file(file_path)

# Process the logs
# ...

# Example: Print the logs
for log in logs[0:10]:
    print(log)

In [ ]:
import ast

import pandas as pd


def convert_log_entry(log_entry):
    # List of keys to potentially convert
    keys_to_convert = ["reward", "info", "observations", "scores"]

    for key in keys_to_convert:
        if key in log_entry and isinstance(log_entry[key], str):
            try:
                log_entry[key] = ast.literal_eval(log_entry[key])
            except (ValueError, SyntaxError):
                # If conversion fails, keep the original string
                pass

    return log_entry


# Convert log entries
converted_logs = [convert_log_entry(log) for log in logs]

# Create DataFrame
df = pd.DataFrame(converted_logs)

# Print column names to see what we're working with
print(df.columns)

# If 'scores' is a column, try to access it
if "scores" in df.columns:
    print(df["scores"].iloc[0])
else:
    print("No 'scores' column found in the DataFrame")

In [ ]:
df[df["message"] == "ddpg"].head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Remove rows where message contains "Connected"
df = df[~df["message"].str.contains("Connected", na=False)]

# Extract the '0' element from scores
df["agent_0_score"] = df["reward"].apply(
    lambda x: x[0] if isinstance(x, dict) else np.nan
)

# Create a function to calculate rolling average


def rolling_average(data, window=10):
    return data.rolling(window=window, min_periods=1).mean()


# Group by message (algorithm type), remove duplicates within each group, and calculate rolling average
grouped = df.groupby("message")
rolling_averages = {}
for name, group in grouped:
    # Remove duplicates based on the episode, keeping the last occurrence
    group = group.drop_duplicates(subset=["episode"], keep="last").sort_values(
        "episode"
    )
    rolling_averages[name] = rolling_average(group["agent_0_score"])

# Create episode numbers
max_episodes = max(len(v) for v in rolling_averages.values())
episodes = range(max_episodes)

# Set up the plot
plt.figure(figsize=(12, 6))

# Colors for each algorithm (extend this dictionary if needed)
colors = {
    "ddqn": "blue",
    "ddpg": "red",
    "maddpg": "green",
    "random": "lightgrey",
    "baseline": "lightblue",
}

# Plot each algorithm
for algo, scores in rolling_averages.items():
    color = colors.get(
        algo.lower(), "gray"
    )  # Use 'gray' if algorithm not in colors dict
    plt.plot(episodes[: len(scores)], scores, label=algo.upper(), color=color)

# Customize the plot
plt.title("Average Performance of Agent 0 over Last 10 Episodes")
plt.xlabel("Episode")
plt.ylabel("Average Score")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Calculate average scores and standard deviations across all episodes for each algorithm
performance_stats = {
    name: (group["agent_0_score"].mean(), group["agent_0_score"].std())
    for name, group in grouped
}

# Sort the results descendingly based on mean score
performance_stats = dict(
    sorted(performance_stats.items(), key=lambda item: item[1][0], reverse=True)
)

# Print the results
print("Algorithm Performance (Mean ± Standard Deviation):")
for algo, (mean, std) in performance_stats.items():
    print(f"{algo.upper()}: {mean:.2f} ± {std:.2f}")

# Calculate the coefficient of variation (CV) to compare variability across algorithms
cv_stats = {
    algo: (std / mean) * 100 if mean != 0 else float("inf")
    for algo, (mean, std) in performance_stats.items()
}

print("\nCoefficient of Variation (lower is more consistent):")
for algo, cv in sorted(cv_stats.items(), key=lambda item: item[1]):
    print(f"{algo.upper()}: {cv:.2f}%")

In [ ]:
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming your DataFrame is called 'df'
# and it has columns 'message' for the algorithm type and 'scores' for the scores

# Define a function to safely extract the '0' element from scores


def extract_agent_0_score(x):
    if isinstance(x, dict):
        return x.get("0", np.nan)
    elif isinstance(x, str):
        try:
            return ast.literal_eval(x).get("0", np.nan)
        except:
            return np.nan
    elif isinstance(x, (int, float)):
        return x
    else:
        return np.nan


# Extract the '0' element from scores
df["agent_0_score"] = df["scores"].apply(extract_agent_0_score)

# Create a function to calculate rolling average


def rolling_average(data, window=10):
    return data.rolling(window=window, min_periods=1).mean()


# Group by message (algorithm type) and calculate rolling average
grouped = df.groupby("message")
rolling_averages = {}
for name, group in grouped:
    rolling_averages[name] = rolling_average(group["agent_0_score"])

# Create episode numbers
max_episodes = max(len(v) for v in rolling_averages.values())
episodes = range(max_episodes)

# Set up the plot
plt.figure(figsize=(12, 6))

# Colors for each algorithm
colors = {"ddqn": "blue", "ddpg": "red", "maddpg": "green"}

# Plot each algorithm
for algo, scores in rolling_averages.items():
    plt.plot(episodes[: len(scores)], scores, label=algo.upper(), color=colors[algo])

# Customize the plot
plt.title("Average Performance of Agent 0 over Last 10 Episodes")
plt.xlabel("Episode")
plt.ylabel("Average Score")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()